## Exploring the Ikeda method
Reimplementing the Matlab code (From Carl-Johan) to python.
Calculation of roll damping for **S175** hull.

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
#plt.style.use('paper')
#plt.style.use('presentation')

#from pylab import rcParams
#rcParams['figure.figsize'] = 15, 5
#matplotlib.matplotlib_fname()

In [ ]:
matplotlib.get_configdir()

In [ ]:
#matplotlibrc_path='../../../rolldecay/matplotlibrc'
#rcParams.update(matplotlib.rc_params_from_file(matplotlibrc_path))

In [ ]:
%matplotlib notebook
#%matplotlib inline

%load_ext autoreload
%autoreload 2

In [ ]:
#from jupyterthemes import jtplot
#jtplot.style(theme='onedork', context='notebook', ticks=True, grid=False)

In [ ]:
import numpy as np
from numpy import sqrt,exp,tanh, pi

import pandas as pd

import rolldecayestimators.ikeda_speed as ikeda
from rolldecay import paper_writing

In [ ]:
#paper_writing.setup(rcParams)

## S175

In [ ]:
visc =   1.15*10**-6  # [m2/s], kinematic viscosity 
g    =   9.81
ra   = 1025          # density of water

Cb   =   0.58        # Block coeff
L    =   175         # Length   
vcg  =   9.52        # roll axis (vertical centre of gravity) [m]
B    =   25.40       # Breadth of hull [m]
d    =   9.5         # Draught of hull [m] 
A    =   0.95*B*d    # Area of cross section of hull [m2] 
bBK  =   0.4         # breadth of Bilge keel [m] !!(=height???)
R    =   3           # Bilge Radis


OG = vcg-d           # distance from roll axis to still water level
Ho = B/(2*d)         # half breadth to draft ratio

LBK  = L/4           # Approx
disp = L*B*d*Cb      # Displacement  
ND_factor = sqrt(B/(2*g))/(ra*disp*(B**2))   # Nondimensiolizing factor of B44

T = 20
wE = 2 * np.pi * 1 / T  # circular frequency
fi_a = 5 * pi / 180;  # roll amplitude !!rad??

In [ ]:
V = np.arange(0,17+1)

Bw0 = ikeda.Bw0_S175(w=wE)
Fn=V/sqrt(g*L)
inputs = pd.DataFrame(index=Fn)
inputs['V']=V
inputs['w']=wE
inputs['fi_a']=fi_a
inputs['Bw0']=Bw0
inputs['d']=d
inputs['B']=B
inputs['A']=A
inputs['bBK']=bBK
inputs['R']=R
inputs['OG']=OG
inputs['Ho']=Ho
inputs['ra']=ra
inputs['Cb']=Cb
inputs['L']=L
inputs['LBK']=LBK
inputs['visc']=visc
inputs['g']=g

results = inputs.apply(func=ikeda.calculate_B44_series, axis=1)

fig,ax=plt.subplots()
result_nondim = results*ND_factor
result_nondim.plot.area(y = ['B_BK','B_L','B_W','B_F'], ax=ax);
ax.set_title('Roll damping components for S175 with original Ikeda method')
ax.set_ylabel('Nondimensional damping')
ax.set_xlabel('$F_n$ [-]');

## Compare with Simplified Ikeda method

In [ ]:
result_nondim.iloc[0]

In [ ]:
from rolldecayestimators import simplified_ikeda

In [ ]:
LPP = L
Beam = B
CB = Cb
CMID = A / (B * d)
OG = OG
PHI = np.rad2deg(fi_a)
lBK=LBK
bBK = bBK
OMEGA = wE
DRAFT = d
B44HAT, BFHAT, BWHAT, BEHAT, BBKHAT, BLHAT = simplified_ikeda.calculate_roll_damping(LPP,Beam,CB,CMID,OG,PHI,
                        lBK,bBK,OMEGA,DRAFT,KVC=visc)

result_simplified = pd.Series(name=0)
result_simplified['B_44']=B44HAT
result_simplified['B_W']=BWHAT
result_simplified['B_F']=BFHAT
result_simplified['B_BK']=BBKHAT
result_simplified['B_E']=BEHAT
result_simplified['B_L']=0
result_simplified['method']='simplified'


In [ ]:
df_compare = pd.DataFrame()
s = result_nondim.iloc[0].copy()
s['method']='ikeda'
df_compare = df_compare.append(s, ignore_index=True)
df_compare = df_compare.append(result_simplified, ignore_index=True)


In [ ]:
fig,ax=plt.subplots()
df_compare.pivot_table(index='method').transpose().plot(kind='bar', ax=ax)



## Faust

In [ ]:
ScaleF =  1 #%/29.565;                  # Scale Factor [-]
visc =   1.15*10**-6;                  # [m2/s], kinematic viscosity
Cb   =   0.61;                        # Block coeff
L    =   220*ScaleF;                  # Length
vcg  =   14.4*ScaleF;                 # roll axis (vertical centre of gravity) [m]
vcg  =   14.9*ScaleF;                 # roll axis (vertical centre of gravity) [m]
B    =   32.26*ScaleF;                # Breadth of hull [m]
d    =   9.5*ScaleF;                  # Draught of hull [m]
A    =   0.93*B*d;                    # Area of cross section of hull [m2]
bBK  =   0.4*ScaleF;                  # breadth of Bilge keel [m] !!(=height???)
R    =   5*ScaleF;                    # Bilge Radis
g    =   9.81;
C_mid=   0.93;

OG = -1*(vcg-d)#*0.8;                    # distance from roll axis to still water level
Ho = B/(2*d);                  # half breadth to draft ratio
ra   = 1025;                   # density of water

#locals
LBK  = L/4;                    # Approx
disp = L*B*d*Cb;               # Displacement

# variables!!
T=27.6*sqrt(ScaleF); 
wE   = 2*pi*1/T;        # circular frequency
fi_a =   10*pi/180;            # roll amplitude !!rad??


ND_factorB = sqrt(B/(2*g))/(ra*disp*(B**2));   # Nondimensiolizing factor of B44
#ND_factor = wE/(2*ra*disp*g*(16*ScaleF-vcg));   # Nondimensiolizing factor of B44

In [ ]:
V = np.arange(0,10+1)

Bw0 = ikeda.Bw0_faust(w=wE)
Fn=V/sqrt(g*L)
inputs = pd.DataFrame(index=Fn)
inputs['V']=V
inputs['w']=wE
inputs['fi_a']=fi_a
inputs['Bw0']=Bw0
inputs['d']=d
inputs['B']=B
inputs['A']=A
inputs['bBK']=bBK
inputs['R']=R
inputs['OG']=OG
inputs['Ho']=Ho
inputs['ra']=ra
inputs['Cb']=Cb
inputs['L']=L
inputs['LBK']=LBK
inputs['visc']=visc
inputs['g']=g

results = inputs.apply(func=ikeda.calculate_B44_series, axis=1)

#plt.rc('text', usetex=True)  
#plt.rc('font', family='serif')
#plt.rc('xtick', labelsize=8)
#plt.rc('ytick', labelsize=8)
#plt.rc('axes', labelsize=8)

fig,ax=plt.subplots()
result_nondim = results*ND_factorB
result_nondim.plot.area(y = ['B_BK','B_L','B_W','B_F'], ax=ax);
ax.set_title('Roll damping components for Faust with original Ikeda method')
ax.set_ylabel('Nondimensional damping')
ax.set_xlabel('$F_n$ [-]');
#fig.set_size_inches(4, 3)
paper_writing.save_fig(fig=fig, name='ikeda_faust')

In [ ]:
LPP = L
Beam = B
CB = Cb
CMID = A / (B * d)
OG = OG
PHI = np.rad2deg(fi_a)
lBK=LBK
bBK = bBK
OMEGA = wE
DRAFT = d
B44HAT, BFHAT, BWHAT, BEHAT, BBKHAT, BLHAT = simplified_ikeda.calculate_roll_damping(LPP,Beam,CB,CMID,OG,PHI,
                        lBK,bBK,OMEGA,DRAFT,KVC=visc)

result_simplified = pd.Series(name=0)
result_simplified['B_44']=B44HAT
result_simplified['B_W']=BWHAT
result_simplified['B_F']=BFHAT
result_simplified['B_BK']=BBKHAT
result_simplified['B_L']=0
result_simplified['B_E']=BEHAT
result_simplified['method']='simplified'

In [ ]:
df_compare = pd.DataFrame()
s = result_nondim.iloc[0].copy()
s['method']='ikeda'
df_compare = df_compare.append(s, ignore_index=True)
df_compare = df_compare.append(result_simplified, ignore_index=True)

In [ ]:
fig,ax=plt.subplots()
df_compare.pivot_table(index='method').transpose().plot(kind='bar', ax=ax)

## Adding Simplified method speed dependancy

In [ ]:
V = np.arange(0,10+1)
Fn=V/sqrt(g*L)
result_simplified = pd.DataFrame(index=Fn)
result_simplified['B_44']=B44HAT
result_simplified['B_W']=BWHAT
result_simplified['B_F']=BFHAT
result_simplified['B_BK']=BBKHAT
result_simplified['B_L']=result_nondim['B_L']
result_simplified['B_E']=BEHAT
result_simplified['method']='simplified'
result_simplified['B_44']+=result_simplified['B_L']

In [ ]:
fig,axes=plt.subplots(nrows=2)
#fig.set_size_inches(7,5)
ax=axes[0]
result_simplified.plot.area(y = ['B_BK','B_L','B_W','B_F','B_E'], ax=ax);
ax.set_title('Roll damping components for Faust with Simplified Ikeda method')
ax.set_ylabel('$\hat{B_{44}}$')
ax.set_ylim(0,0.01)

ax=axes[1]
result_nondim.plot.area(y = ['B_BK','B_L','B_W','B_F'], ax=ax);
ax.set_title('Roll damping components for Faust with original Ikeda method')
ax.set_ylabel('$\hat{B_{44}}$')
ax.set_xlabel('$F_n$ [-]');
plt.tight_layout()
ax.get_legend().remove()
ax.set_ylim(0,0.01)
paper_writing.save_fig(fig=fig, name='ikeda_vs_simplified')



In [ ]:
def simplified_speed(row):
    
    V=row['V']
    B44HAT, BFHAT, BWHAT, BEHAT, BBKHAT, BLHAT = simplified_ikeda.calculate_roll_damping(LPP,Beam,CB,CMID,OG,PHI,
                            lBK,bBK,OMEGA,DRAFT,V=V, KVC=visc)
    
    result_simplified = pd.Series(name=0)
    result_simplified['B_44']=B44HAT
    result_simplified['B_W']=BWHAT
    result_simplified['B_F']=BFHAT
    result_simplified['B_BK']=BBKHAT
    result_simplified['B_L']=BLHAT
    result_simplified['B_E']=BEHAT
    result_simplified['method']='simplified'
    return result_simplified

In [ ]:
result_simplified2 = inputs.apply(func=simplified_speed,axis=1)
fig,ax=plt.subplots()
result_simplified2.plot.area(y = ['B_BK','B_L','B_W','B_F','B_E'], ax=ax);
ax.set_title('Roll damping components for Faust with Simplified Ikeda method')
ax.set_ylabel('Nondimensional damping')
ax.set_xlabel('$F_n$ [-]');